In [205]:
import sys
sys.path.append('../../../src/')

import os
import pandas as pd
import numpy as np
import config
import utils
import torch
import pickle

from Model import Impf_DNAMLP
from torch.nn.functional import softmax

In [206]:
fold = '1.1'
outer_fold = '1.0'
group = 'Sella'

In [207]:
dat_cfg = config.data_config
clf_cfg = config.classifier_config
impf_cfg = config.impf_config
TRAIN_CSV_DIR = dat_cfg['TRAIN_CSV_DIR']
TEST_CSV_DIR = dat_cfg['TEST_CSV_DIR']
OUTER_FOLD_CSV_FILE = os.path.join(TEST_CSV_DIR, group, f'{outer_fold}_test.csv')
device = impf_cfg['device']
device

'cpu'

In [208]:
alg = 'RF'

if alg.lower() == 'rf':
    selector_name = 'RF'
    print('Alg: RF')
elif alg.lower() == 'mlp':
    selector_name = 'MLP'
    print('Alg: MLP')
elif alg.lower() == 'lr':
    print('Alg: LR')
    selector_name = 'LR'
else:
    print('Default alg: RF')
    selector_name = 'RF'
    
with open(os.path.join(impf_cfg['IMPORTANT_FEATURES_DIR'], alg, group, f'{outer_fold}_combined.pkl'), 'rb') as handle:
    impf = pickle.load(handle)

Alg: RF


In [209]:
df_test = pd.read_csv(OUTER_FOLD_CSV_FILE, index_col=0)
df_test.shape

(64, 10002)

In [210]:
pd.read_csv(OUTER_FOLD_CSV_FILE, index_col=0)

,cg22054918,cg16476975,cg03940848,cg25570913,cg02809746,cg01517680,cg04110886,cg18222083,cg05649108,cg08141395,...,cg06075311,cg19145082,cg26691179,cg04842161,cg07621803,cg21100077,cg00703120,cg00540828,label,class_group
5911815023_R04C01,0.003412,0.751280,0.013430,0.015896,0.256333,0.459410,0.437303,0.025581,0.024721,0.810745,...,0.228252,0.573620,0.842861,0.560194,0.879508,0.363078,0.258965,0.598538,"CONTR, CEBM",Control
7310440006_R03C01,0.030944,0.918441,0.688537,0.035199,0.033865,0.797875,0.462733,0.839347,0.795607,0.889551,...,0.264708,0.664471,0.811253,0.712039,0.559203,0.213917,0.152059,0.184225,"CONTR, CEBM",Control
7310440006_R06C01,0.015632,0.030063,0.057383,0.515874,0.101463,0.243184,0.177028,0.569094,0.294695,0.415792,...,0.605413,0.425003,0.233670,0.790648,0.919498,0.614223,0.790126,0.923501,"CONTR, HEMI",Control
8622007069_R04C02,0.121080,0.263024,0.051560,0.281109,0.071821,0.530285,0.501271,0.117821,0.368817,0.177324,...,0.290296,0.271382,0.583402,0.705860,0.795193,0.440586,0.715731,0.930837,"CONTR, REACT",Control
8622007080_R03C02,0.027579,0.030399,0.054074,0.431553,0.130259,0.113425,0.144355,0.501967,0.313516,0.436275,...,0.663769,0.413085,0.229725,0.812363,0.923481,0.673290,0.823718,0.804887,"CONTR, HEMI",Control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999095103_R03C02,0.073762,0.076599,0.234333,0.652253,0.180507,0.318403,0.349878,0.646680,0.506870,0.377636,...,0.579342,0.455967,0.490922,0.701557,0.884962,0.732384,0.814823,0.964649,"CONTR, HEMI",Control
3998523003_R01C01,0.098493,0.119184,0.071567,0.510305,0.249399,0.206461,0.252531,0.451762,0.286400,0.200343,...,0.744708,0.480333,0.289881,0.701547,0.951435,0.437180,0.846317,0.882564,"CONTR, HYPTHAL",Control
3998523003_R03C01,0.157040,0.098117,0.133763,0.572476,0.264706,0.222320,0.286853,0.400981,0.413690,0.308592,...,0.691893,0.429534,0.303310,0.779729,0.943066,0.685437,0.780231,0.938310,"CONTR, HYPTHAL",Control
3998523022_R01C01,0.086842,0.026169,0.046485,0.022780,0.052046,0.077527,0.938776,0.575809,0.833146,0.918189,...,0.840030,0.116124,0.232922,0.070486,0.420369,0.085394,0.851745,0.982588,"PITAD, TSH",Sella


In [211]:
pd.Series(pd.read_csv(OUTER_FOLD_CSV_FILE, index_col=0).iloc[:, -1]).value_counts()

Sella      37
Control    27
Name: class_group, dtype: int64

In [212]:
df_test

,cg22054918,cg16476975,cg03940848,cg25570913,cg02809746,cg01517680,cg04110886,cg18222083,cg05649108,cg08141395,...,cg06075311,cg19145082,cg26691179,cg04842161,cg07621803,cg21100077,cg00703120,cg00540828,label,class_group
5911815023_R04C01,0.003412,0.751280,0.013430,0.015896,0.256333,0.459410,0.437303,0.025581,0.024721,0.810745,...,0.228252,0.573620,0.842861,0.560194,0.879508,0.363078,0.258965,0.598538,"CONTR, CEBM",Control
7310440006_R03C01,0.030944,0.918441,0.688537,0.035199,0.033865,0.797875,0.462733,0.839347,0.795607,0.889551,...,0.264708,0.664471,0.811253,0.712039,0.559203,0.213917,0.152059,0.184225,"CONTR, CEBM",Control
7310440006_R06C01,0.015632,0.030063,0.057383,0.515874,0.101463,0.243184,0.177028,0.569094,0.294695,0.415792,...,0.605413,0.425003,0.233670,0.790648,0.919498,0.614223,0.790126,0.923501,"CONTR, HEMI",Control
8622007069_R04C02,0.121080,0.263024,0.051560,0.281109,0.071821,0.530285,0.501271,0.117821,0.368817,0.177324,...,0.290296,0.271382,0.583402,0.705860,0.795193,0.440586,0.715731,0.930837,"CONTR, REACT",Control
8622007080_R03C02,0.027579,0.030399,0.054074,0.431553,0.130259,0.113425,0.144355,0.501967,0.313516,0.436275,...,0.663769,0.413085,0.229725,0.812363,0.923481,0.673290,0.823718,0.804887,"CONTR, HEMI",Control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999095103_R03C02,0.073762,0.076599,0.234333,0.652253,0.180507,0.318403,0.349878,0.646680,0.506870,0.377636,...,0.579342,0.455967,0.490922,0.701557,0.884962,0.732384,0.814823,0.964649,"CONTR, HEMI",Control
3998523003_R01C01,0.098493,0.119184,0.071567,0.510305,0.249399,0.206461,0.252531,0.451762,0.286400,0.200343,...,0.744708,0.480333,0.289881,0.701547,0.951435,0.437180,0.846317,0.882564,"CONTR, HYPTHAL",Control
3998523003_R03C01,0.157040,0.098117,0.133763,0.572476,0.264706,0.222320,0.286853,0.400981,0.413690,0.308592,...,0.691893,0.429534,0.303310,0.779729,0.943066,0.685437,0.780231,0.938310,"CONTR, HYPTHAL",Control
3998523022_R01C01,0.086842,0.026169,0.046485,0.022780,0.052046,0.077527,0.938776,0.575809,0.833146,0.918189,...,0.840030,0.116124,0.232922,0.070486,0.420369,0.085394,0.851745,0.982588,"PITAD, TSH",Sella


In [213]:
rand = np.random.randint(df_test.shape[0])
sample = df_test.iloc[rand,:].loc[impf]
df_test.iloc[rand, -2:], sample

C:\Users\OA\AppData\Local\Temp/ipykernel_12256/2921419439.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  sample = df_test.iloc[rand,:].loc[impf]


(label          CONTR, REACT
 class_group         Control
 Name: 8622007119_R03C01, dtype: object,
 cg01822130    0.057973
 cg22900415    0.087497
 cg08690999    0.480272
 cg03349020    0.434909
 cg02455706    0.460791
                 ...   
 cg14232291    0.165729
 cg04213900      0.7574
 cg26952362    0.980365
 cg07664029    0.718111
 cg03393769    0.744206
 Name: 8622007119_R03C01, Length: 2095, dtype: object)

In [214]:
sample_features = np.expand_dims(np.array(sample).astype(float), axis = 0)
sample_features, type(sample_features), sample_features.shape, type(sample_features[0][0])

(array([[0.05797284, 0.08749671, 0.48027198, ..., 0.98036457, 0.7181109 ,
         0.74420613]]),
 numpy.ndarray,
 (1, 2095),
 numpy.float64)

In [215]:
IMPF_MLP_STATE_FILE = os.path.join(impf_cfg['MLP_BEST_STATES_DIR'], alg, group, f'{fold}.pth')

in_features = impf_cfg['N_IMPF']
n_classes = impf_cfg['n_classes']
in_features = len(impf)

model = Impf_DNAMLP(in_features, n_classes)
model.load_state_dict(torch.load(IMPF_MLP_STATE_FILE))
model.to(device)

Impf_DNAMLP(
  (densenet): Sequential(
    (0): Linear(in_features=2095, out_features=2, bias=True)
  )
)

In [216]:
sample_features.shape

(1, 2095)

In [217]:
logits = softmax(model((torch.Tensor(sample_features).to(device))), dim = 1)
logits

tensor([[0.1068, 0.8932]], grad_fn=<SoftmaxBackward0>)